In [1]:
# ==========================================
# ІМПОРТ БІБЛІОТЕК [cite: 86, 91]
# ==========================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.datasets import make_regression

print(f"TensorFlow Version: {tf.__version__}")


def plot_history(history, title="Model Training"):
    plt.figure(figsize=(12, 4))
    
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    
    if 'accuracy' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{title} - Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
    
    plt.show()

# ==========================================
# ЧАСТИНА 1: РЕГРЕСІЯ (Neural Network) [cite: 87, 102]
# ==========================================
print("\n=== ЧАСТИНА 1: РЕГРЕСІЯ ===")


X_reg, y_reg = make_regression(n_samples=1000, n_features=1, noise=15, random_state=42)


X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

scaler_reg = StandardScaler()
X_train_reg = scaler_reg.fit_transform(X_train_reg)
X_test_reg = scaler_reg.transform(X_test_reg)


model_reg = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_reg.shape[1],)), 
    layers.Dense(32, activation='relu'),                                     
    layers.Dense(1)                                                         
])


model_reg.compile(optimizer='adam', loss='mse', metrics=['mae'])


print("Навчання моделі регресії...")
history_reg = model_reg.fit(X_train_reg, y_train_reg, epochs=50, batch_size=32, validation_split=0.2, verbose=0)


loss_reg, mae_reg = model_reg.evaluate(X_test_reg, y_test_reg, verbose=0)
print(f"Regression MAE: {mae_reg:.4f}")

plot_history(history_reg, "Regression")


y_pred_reg = model_reg.predict(X_test_reg)
plt.figure(figsize=(8, 6))
plt.scatter(X_test_reg, y_test_reg, color='blue', alpha=0.5, label='Actual Data')
plt.scatter(X_test_reg, y_pred_reg, color='red', alpha=0.5, label='NN Predictions')
plt.title("Regression: Actual vs Predicted")
plt.legend()
plt.show()

# ==========================================
# ЧАСТИНА 2: КЛАСИФІКАЦІЯ (Bats Dataset) [cite: 87, 103]
# ==========================================
print("\n=== ЧАСТИНА 2: КЛАСИФІКАЦІЯ (Bats) ===")

try:
 
    df = pd.read_csv('bats_information.csv')
    df = df.dropna()
    
    target_col = 'Species'
    
    
    X_raw = df.drop(target_col, axis=1)
    y_raw = df[target_col]
    
  
    X_encoded = pd.get_dummies(X_raw, drop_first=True)
    
   
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y_raw)
    y_categorical = tf.keras.utils.to_categorical(y_encoded) 
    
    num_classes = y_categorical.shape[1]
    
   
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_encoded, y_categorical, test_size=0.2, random_state=42)
    
    scaler_c = StandardScaler()
    X_train_c = scaler_c.fit_transform(X_train_c)
    X_test_c = scaler_c.transform(X_test_c)
    
   
    model_class = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_c.shape[1],)),
        layers.Dropout(0.3), 
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax') 
    ])
    
   
    model_class.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    print("Навчання моделі класифікації...")
   
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    
    history_class = model_class.fit(X_train_c, y_train_c, 
                                    epochs=100, 
                                    batch_size=16, 
                                    validation_split=0.2, 
                                    callbacks=[callback],
                                    verbose=0)
    
   
    loss_c, acc_c = model_class.evaluate(X_test_c, y_test_c, verbose=0)
    print(f"Classification Accuracy: {acc_c:.4f}")
    
    plot_history(history_class, "Classification")
    
    
    y_pred_probs = model_class.predict(X_test_c)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_true_classes = np.argmax(y_test_c, axis=1)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_matrix(y_true_classes, y_pred_classes), annot=True, fmt='d', cmap='Blues')
    plt.title("Confusion Matrix (Neural Network)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

except FileNotFoundError:
    print("⚠️ Файл 'bats_information.csv' не знайдено. Завантажте його в середовище.")
except Exception as e:
    print(f"Сталася помилка: {e}")

ModuleNotFoundError: No module named 'tensorflow.python'